In [1]:
using DifferentialEquations
using Plots


In [2]:
function laplacian(u, dx)
    n = length(u)
    du = similar(u)
    du[1] = (u[2] - 2u[1] + u[n-1]) / dx^2
    du[n] = (u[1] - 2u[n] + u[n-1]) / dx^2
    for i in 2:n-1
        du[i] = (u[i+1] - 2u[i] + u[i-1]) / dx^2
    end
    return du
end

function laplacian_2D(u, dx)
    n, m = size(u)
    du = zeros(size(u))
    for i in 1:n
        du[i, :] += laplacian(u[i, :], dx)
    end
    for i in 1:m
        du[:, i] += laplacian(u[:, i], dx)
    end
    du
end

function reaction_diffusion!(du, u, p, t)
    dx, D1, D2,f, g = p
    u0, u1 = u[:, :, 1], u[:, :, 2]
    dif_u0 = laplacian_2D(u0, dx)
    dif_u1 = laplacian_2D(u1, dx)
    
    du[:, :, 1] = D1 .* dif_u0 .+ f(u0, u1)
    du[:, :, 2] = D2 .* dif_u1 .+ g(u0, u1)
end

"""
function solve_diffeq(u0, grad_fn, dt, nsteps)
    p = (dx, D1, D2, f, g)
    dt = 0.1
    du = similar(u0)
    for i in 1:nsteps
        grad_fn(du, u0, p, 0)
        u0 += dt * du
    end
    u0
end
"""

"function solve_diffeq(u0, grad_fn, dt, nsteps)\n    p = (dx, D1, D2, f, g)\n    dt = 0.1\n    du = similar(u0)\n    for i in 1:nsteps\n        grad_fn(du, u0, p, 0)\n        u0 += dt * du\n    end\n    u0\nend\n"

In [3]:
function run_gray_scott(initialization="random")
    D1 = 2e-5
    D2 = 1e-5
    dx = .01

    k0 = 0.022
    k1 = 0.051

    f = (u, v) -> -u .* v.^2 + k0 .* (1 .- u)
    g = (u, v) -> u .* v.^2 - (k0 + k1) .* v


    grid_size = 256
    u0 = zeros(grid_size, grid_size, 2)
    if initialization == "square"
        u0[100:156, 100:156, 1] .= .9 
        u0[100:156, 100:156, 2] .= .1 
    elseif initialization == "random"
        u0[:, :, 1] .+= .95 .+ rand(grid_size, grid_size) * 0.1
        u0[:, :, 2] .+=  rand(grid_size, grid_size) * 0.2
    end
    
    p = (dx, D1, D2, f, g)
    tspan = (0.0, 3000.)
    prob = ODEProblem(reaction_diffusion!, u0, tspan, p)
    sol = solve(prob)
    sol
end


run_gray_scott (generic function with 2 methods)

In [14]:
sol = run_gray_scott("square")
print("")

In [15]:
for i in 1:length(sol)
    if (rem(i, 500) == 1) | (i == length(sol))
        p = heatmap(sol[i][:,:,1], title="t=$(sol.t[i])")
        savefig(p, "figures/step_$(i).png")
    end
end